## Train an *E*FN using the `energyflow` package

In [1]:
import numpy as np
import tensorflow.keras.backend as K
import energyflow as ef
from energyflow.archs import EFN

from data import get_data

2023-08-30 04:22:15.972687: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-30 04:22:17.855338: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
task_name = "scalar1"

In [3]:
(X_train, X_val, X_test,
 Y_train, Y_val, Y_test) = get_data(task_name)

z_train, z_val, z_test = X_train[:,:,2], X_train[:,:,2], X_train[:,:,2]
p_train, p_val, p_test = X_train[:,:,[0,1,3]], X_train[:,:,[0,1,3]], X_train[:,:,[0,1,3]]

In [4]:
Phi_sizes = (256,) * 4 + (128,) * 4
F_sizes = (256,) * 4 + (128,) * 4
pfn = EFN(
    input_dim=p_train.shape[-1],
    output_dim=3,
    Phi_sizes=Phi_sizes,
    F_sizes=F_sizes
)

2023-08-30 04:22:43.141714: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1635] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 15363 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:07:00.0, compute capability: 6.0


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 phats_input (InputLayer)       [(None, None, 3)]    0           []                               
                                                                                                  
 tdist_0 (TimeDistributed)      (None, None, 256)    1024        ['phats_input[0][0]']            
                                                                                                  
 activation (Activation)        (None, None, 256)    0           ['tdist_0[0][0]']                
                                                                                                  
 tdist_1 (TimeDistributed)      (None, None, 256)    65792       ['activation[0][0]']             
                                                                                              

In [6]:
num_epochs = 45
batch_size = 100
K.set_value(pfn.model.optimizer.learning_rate, 2e-4)

pfn.fit([z_train, p_train], Y_train,
        epochs=num_epochs,
        batch_size=batch_size,
        validation_data=([z_test, p_test], Y_test),
        verbose=1)

Epoch 1/45
2100/2100 [==============================] - ETA: 0s - loss: 201.3618 - acc: 0.4109

ValueError: Data cardinality is ambiguous:
  x sizes: 210000, 210000
  y sizes: 90000
Make sure all arrays contain the same number of samples.

In [ ]:
num_epochs = 45
batch_size = 100
K.set_value(pfn.model.optimizer.learning_rate, 2e-5)

pfn.fit([z_train, p_train], Y_train,
        epochs=num_epochs,
        batch_size=batch_size,
        validation_data=([z_val, p_val], Y_val),
        verbose=1)

In [ ]:
num_epochs = 45
batch_size = 100
K.set_value(pfn.model.optimizer.learning_rate, 2e-6)

pfn.fit([z_train, p_train], Y_train,
        epochs=num_epochs,
        batch_size=batch_size,
        validation_data=([z_val, p_val], Y_val),
        verbose=1)

In [ ]:
model.save(f"/usatlas/atlas01/atlasdisk/users/atlas_wifeng/photon-jet/models/{task_name}_efn_lib")